# Notebook 4: COMPAS Case Study of Algorithmic Fairness

In this notebook, we'll take a look at questions of fairness in algorithmic decision making and risk assessment systems as we analyze COMPAS—a commercial tool used to inform sentencing by predicting an individual's risk of recidivism (the likelihood that a convicted person will commit another crime) on a scale of 1-10. 

COMPAS is <i>the</i> case study of recidivism risk-scoring algorithms. Understanding COMPAS and the conversation around it is essential knowledge for justice-oriented data thinkers like ourselves. **If you aren't already familiar with COMPAS, take a moment to read [ProPublica's "Machine Bias;"](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing) which was a recommended reading in week nine.**

### Learning Objectives
* Identify and explain how algorithms exist as technologies that interact with humans in social institutions, often in nontrivial ways.
* Recognize that fairness is a socially-defined and dynamic idea.
* Understand how definitions of fairness can be used to evaluate algorithms and come to distinct conclusions.
* Recognize the limitations of technical solutions to algorithmic bias with social consequences.

### Table of Contents
1. [Understanding COMPAS](#1.-Understanding-COMPAS)
2. [The Debate about COMPAS](#2.-The-Debate-about-COMPAS)
3. [ProPublica's Analysis](#3.-ProPublica's-Analysis)
4. [equivant's Analysis](#4.-equivant's-Analysis)
5. [Conclusion](#5.-Conclusion)


Before we begin, run the cell below to install necessary dependencies.

In [ ]:
# Just run this cell
import numpy as np # Loads numerical methods
import pandas as pd # This allows us to transform our data into tables (AKA DataFrames) 

from sklearn.metrics import roc_curve, roc_auc_score # Loads visualization tools for a ROC Curve
import matplotlib.pyplot as plt # Loads more visualization tools
import seaborn as sns # Loads more visualization tools

import otter
generator = otter.Notebook()

## 1. Understanding COMPAS

### What is COMPAS?
COMPAS (Correctional Offender Management Profiling for Alternative Sanctions) is a commercial tool originally produced by the for-profit company Northpointe, that has since been acquired by the also for-profit [equivant](https://www.equivant.com). We will refer to the company behind COMPAS as equivant, but conversations you encounter online may have been written before the acquistion, and still refer to Northpointe (such as the ProPublica article).

Decision making within the US criminal justice system relies heavily on risk assessment, especially in conversations of pre-trial detainment and recidivism. Risk has become the language with which a judge or parole board formulates and justifies their decision to set bail, grant parole, and determine sentencing, based on their beliefs of the likelihood that a defendant will fail to appear in court, commit another crime, or pose a threat to the public. While the concept of labeling defendants as more or less "risky," isn't new in the justice system, the use of risk scores determined by algorithms to inform these labels is gaining traction.

COMPAS was originally designed to aid decisions made by probation officers on treatment and supervision of incarcerated people, but it and other risk-scoring algorithms are being used to inform decisions about freedom at numerous points throughout the criminal justice system. It does this by producing a risk score from 1 (lowest) to 10 (highest) based on the model's prediction of the likelihood a defendant will commit a future crime.

#### Language Matters
In this conversation, we deal with terms like **model**, **algorithm**, **bias**, and **risk**. These terms often have different definitions and implications in data science contexts than they do in our daily conversations.

In data science and machine learning, COMPAS is what we refer to as a **model**. A model is composed of a dataset that has **features** (the columns in a table that categorize its data) and an **algorithm**—a procedure that makes a prediction about unknown values based on patterns found in the data. 

<!-- BEGIN QUESTION -->
<div class="alert alert-block alert-info">
<b>Question 1a:</b> How do you use the term "algorithm" in your daily life? Is it different than the way it is used in technical conversations?
</div>

<!--
BEGIN QUESTION
name: q1a
points: 1
manual: true
-->

**Answer:** *Double-click to edit with your answer*

<!-- END QUESTION -->

Similarly, in data science, **risk** and **bias** are terms used to describe how well a model performs the task it was designed for. **Risk functions** quantify how accurate a model's predictions are, and help us reduce error. Bias is a type of error that describes model inaccuracy as a result of our model systematically misrepresenting the real world phenomena it is designed to capture and can be minimized using better math and engineering. This is a different way of thinking about bias than we are used to in conversations about society, discrimination, and injustice.
    
<!-- BEGIN QUESTION -->
<div class="alert alert-block alert-info">
<b>Question 1b:</b> How is the term bias used in conversations about technology and justice systems? Is it used differently in conversations about data science and machine learning?
</div>
 
<!--
BEGIN QUESTION
name: q1b
points: 1
manual: true
-->

**Answer:** *Double-click to edit with your answer*

<!-- END QUESTION -->

In the real world, bias, as it is communly used, cannot be eliminated or solved for. For that reason, its sometimes more helpful to think of bias as **perspective** in conversations about justice. Doing so can remind us that the lens with which we view the world, informed by our position within it, inevitably distorts what we see and the conclusions we draw about it. Our perspectives are unique and everpresent, and cannot be solved away or minimized like we might treat a bias term in a data science context.
    
<!-- BEGIN QUESTION -->
<div class="alert alert-block alert-info">
<b>Question 1c:</b> Identify three parties that are impacted by COMPAS. How are they impacted? Can you think of at least one example of impacts beyond the scope of the courtroom?
</div>
 
<!--
BEGIN QUESTION
name: q1c
points: 1
manual: true
-->

**Answer:** *Double-click to edit with your answer*

<!-- END QUESTION -->

#### How COMPAS works
Before COMPAS can create a risk score for a defendant, they are asked to fill out a questionnaire. Let's take a look at this questionnaire to get a better understanding of what goes into determining a risk score.

Take a look at [this sample questionnaire](https://www.documentcloud.org/documents/2702103-Sample-Risk-Assessment-COMPAS-CORE.html), used by COMPAS.

<!-- BEGIN QUESTION -->
<div class="alert alert-block alert-info">
<b>Question 1d:</b> What aspects of the questionnaire were particularly striking to you?
</div>

<!--
BEGIN QUESTION
name: q1d
points: 1
manual: true
-->

**Answer:** *Double-click to edit with your answer*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->
<div class="alert alert-block alert-info">
<b>Question 1e:</b> Which stories about criminality and justice does this questionairre reinforce or reinvent? Are those stories different than the ones you are used to hearing? Identify one or more readings we've done in this class that can help you understand the relationship between COMPAS and stories of criminality.
</div>

<!--
BEGIN QUESTION
name: q1e
points: 1
manual: true
-->

**Answer:** *Double-click to edit with your answer*

<!-- END QUESTION -->

## 2. The Debate about COMPAS

Proponents of COMPAS and other risk-assessment tools argue that risk scores help judges make more accurate, consistent, and informed decisions. Algorithms are often seen as evening out the emotions and imperfections that pervade human judgement ([Henry](https://theappeal.org/risk-assessment-explained/)). Opponents disagree, shedding doubt on the algorithm's accuracy and criticizing its potential to exacerbate discrimination in courtrooms.

For the rest of the notebook, we'll replicate the analyses used by equivant and ProPublica to understand how competing definitions of fairness can be mathematically incompatible.

#### What is ProPublica?
ProPublica is a nonprofit organization that, according to their website, "produces investigative journalism with moral force” ([ProPublica](https://www.propublica.org/about/)). ProPublica was founded as a nonpartisan newsroom aiming to expose and question abuses of power, justice, and public trust, often by systems and institutions deeply ingrained in the US.

In 2016, ProPublica investigated the COMPAS algorithm to assess its accuracy and guage the potential of racial bias within the tool. In their analysis, ProPublica used data from defendants given risk scores from Broward County, Florida from 2013 and 2014 to test for statistical differences in outcomes for Black and white defendants. ProPublica came to the conclusion that COMPAS treats Black defendants unfairly because the model "was particularly likely to falsely flag black defendants as future criminals, wrongly labeling them this way at almost twice the rate as white defendants" ([ProPublica](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing)). ProPublica claimed that this was a result of the model utilizing data from a criminal justice system with a history of racial injustices, thus continuing to disproportionately target and arrest Black people in comparison to their white counterparts. 

#### What is equivant?

equivant (then Northpointe) refuted the claim that COMPAS treated Black defendants unfairly. Northpointe was a for-profit computer software company that aimed to advance justice by informing and instilling confidence in decision makers at every stage of the criminal justice system ([equivant](https://www.equivant.com/)).  In addition to operating and continuing to develop COMPAS, equivant has developed a variety of technologies for use in court case management, attorney case management, inmate classification, and risk/needs assessment strategies.  

In the wake of criticism from ProPublica and other researchers, equivant produced a [detailed response](http://go.volarisgroup.com/rs/430-MBX-989/images/ProPublica_Commentary_Final_070616.pdf) to ProPublica’s allegations, claiming that these critiques of their tool utilized the wrong type of classification statistics in their analysis and portrayed the tool incorrectly. The company provided their own analysis of the COMPAS algorithm by using different statistical methods and responding individually to each of ProPublica’s claims of racial bias against Black defendants. 

Upon examining their tool’s fairness through "*accuracy equity*" (a metric coined by equivant that was left out of ProPublica’s analysis), as well as the fact that the model was not trained with a race feature, equivant concluded that their algorithm treats all people and specified groups equally, and therefore does not exhibit signs of bias or inequality for specified groups.

#### Competing definitions of fairness

ProPublica claimed that COMPAS was unfair for Black defendants because it falsely predicted that Black defendants would commit future crimes at nearly twice the rate that it made the same error for white defendants. Put another way, ProPublica argued that an algorithm cannot be fair if it makes errors in predicting recidivism rates for one racial group more frequently than another.

equivant claimed that COMPAS *was* fair because Black and white defendants were discriminated against equally—the score outputted corresponded to the same outcomes irrespective of race. They pointed to data that showed Black defendants and white defendants, assigned the same risk score, went on to commit future crimes at nearly the same rates.

In what follows, we'll retrace both ProPublica and equivant's statistical analysis in order to better understand their arguments and the metrics of fairness they use. By the time we're done, we'll have demonstrated that definitions of fairness can be internally sound, but mathematically incompatible with each other.

## 3. ProPublica's Analysis
### Disproportionate risk scores
To understand why ProPublica found COMPAS to be unfair, we'll look at the same data they used to conduct their analysis—a collection of public records containing people assigned risk scores from 2013 and 2014 in Broward County, Florida ([ProPublica](https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm)).

In [ ]:
# Run this cell to load the data used by ProPublica
propublica_data = pd.read_csv("data/compas-scores-two-years.csv")
propublica_data

ProPublica explains some of the process by which they filtered the data, stating, "It was not always clear, however, which criminal case was associated with an individual’s COMPAS score. To match COMPAS scores with accompanying cases, we considered cases with arrest dates or charge dates within 30 days of a COMPAS assessment being conducted. In some instances, we could not find any corresponding charges to COMPAS scores. We removed those cases from our analysis" ([ProPublica](https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm)). 

In the cell below, we do the same filtering, and narrow down the features of the dataset to those relevant to ProPublica's analysis.

In [ ]:
# Just run this cell
propublica_data = propublica_data.query('days_b_screening_arrest <= 30 & days_b_screening_arrest >= -30')

propublica_data = propublica_data[["age", "c_charge_degree", "race", "age_cat", "score_text", "sex", "priors_count", 
                    "days_b_screening_arrest", "decile_score", "is_recid", "two_year_recid"]]
propublica_data

<!-- BEGIN QUESTION -->
<div class="alert alert-block alert-info">
<b>Question 3a:</b> Explore the dataset. What does a row in the dataset represent? 
</div>

<!--
BEGIN QUESTION
name: q3a
points: 1
manual: true
-->

**Answer:** *Double-click to edit with your answer*

<!-- END QUESTION -->

#### Visualizing risk scores
Let's visualize how ProPublica began their investigation of racial disparity within the COMPAS risk assessment. The histogram below displays the differences between risk scores (the 'decile_score' feature in the data above) assigned to Black and white defendants.

In [ ]:
# Run this cell
white_defendant_scores = propublica_data[propublica_data["race"] == "Caucasian"]
sns.histplot(white_defendant_scores["decile_score"], label="White defendants")

black_defendant_scores = propublica_data[propublica_data["race"] == "African-American"]
sns.histplot(black_defendant_scores["decile_score"], label="Black defendants"); 

plt.xlabel("COMPAS Assigned Risk Score")
plt.legend()
plt.show()

<!-- BEGIN QUESTION -->
<div class="alert alert-block alert-info">
<b>Question 3b:</b> Examine the visualization above. Is one racial group more likely to recieve higher or lower risk scores than the other? Explain how you know.
</div>

<!--
BEGIN QUESTION
name: q3b
points: 1
manual: true
-->

**Answer:** *Double-click to edit with your answer*

<!-- END QUESTION -->

### False Positive and False Negative Rates

In order to understand the ways in which bias is present in COMPAS's assigned risk scores, ProPublica used the **False Positive Rate (FPR)** and **False Negative Rate (FNR)** as their metrics to understand and quantify fairness. Recall that models use algorithms to predict a target feature (in this case the likelihood of recidivism). We can evaluate how well a model predicts these features by using metrics like **false positive** and **false negative** rates (as well as their counterparts, true positive and true negative rates).

A false positive occurs when a model incorrectly predicts an positive outcome. For instance, testing positive for COVID-19 when you don't actually have the virus. A false negative follows similar logic, occuring when a model incorrectly predicts a negative outcomes. For instance, testing negative for COVID-19 when you do infact have the virus.

It's important to remember that the cost of false positives and false negatives isn't usually the same. To extend the COVID-19 example, it is perhaps favorable to design a test that has a higher false positive rate in exchange for a lower false negative rate. Falsely testing positive for COVID-19 usually results in an individual taking another test or going to the doctor to confirm the first test. Falsely testing negative for COVID-19, on the other hand, could have more serious ramifications if the individual goes on to expose others to the virus unknowingly.

<!-- BEGIN QUESTION -->
<div class="alert alert-block alert-info">
<b>Question 3c:</b> Remember that the goal of COMPAS is to predict whether or not a defendant will commit a future crime. In the context of COMPAS, what would a false positive be? What would a false negative be?
</div>

<!--
BEGIN QUESTION
name: q3c
points: 1
manual: true
-->

**Answer:** *Double-click to edit with your answer*

<!-- END QUESTION -->

Below, we define functions to calculate the false positive and false negative rates based on the data from ProPublica. In the data, the 'two_year_recid' feature contains a 0 if the individual did not commit another crime in two years since they were assigned a risk score, and a 1 otherwise. COMPAS predicts that a defendant will commit a future crime if they receive a risk score greater than or equal to 4. As such, a false positive occurs when a defendant receieves a 'decile_score' of 4 or greater, but has a 'two_year_recid' value of 0.

In [ ]:
# Just run this cell
def fpr(race_feature, data):
    # Function returns the False Positive Rate of scores for the specified race_feature
    
    subgroup = data[data["race"] == race_feature]
    did_not_recidivate = subgroup[subgroup["two_year_recid"] == 0]

    false_positives = did_not_recidivate[did_not_recidivate["decile_score"] > 4].shape[0]
    true_negatives = did_not_recidivate[did_not_recidivate["decile_score"] <= 4].shape[0]
    return false_positives / (false_positives + true_negatives)


def fnr(race_feature, data):
    # Function returns the False Negative Rate of scores for the specified race_feature
    
    subgroup = data[data["race"] == race_feature]
    recidivated = subgroup[subgroup["two_year_recid"] == 1]
    
    false_negatives = recidivated[recidivated["decile_score"] <= 4].shape[0]
    true_positives = recidivated[recidivated["decile_score"] > 4].shape[0]
    return false_negatives / (false_negatives + true_positives)

In [ ]:
# Here we use the functions we just defined to calculate the FPR/FNR for Black and white defendants
# Just run this cell
black_defendant_fpr = round(fpr("African-American", propublica_data), 5) * 100
white_defendant_fpr = round(fpr("Caucasian", propublica_data), 5) * 100
black_defendant_fnr = round(fnr("African-American", propublica_data), 4) * 100
white_defendant_fnr = round(fnr("Caucasian", propublica_data), 5) * 100

print("FPR for Black defendants:", black_defendant_fpr, "%")
print("FPR for white defendants:", white_defendant_fpr, "%")
print("FNR for Black defendants:", black_defendant_fnr, "%")
print("FNR for white defendants:", white_defendant_fnr, "%")

<!-- BEGIN QUESTION -->
<div class="alert alert-block alert-info">
<b>Question 3d:</b> What do you notice about these percentages? Is COMPAS making incorrect predictions for one racial group more than the other? Which kinds of predictions?
</div>

<!--
BEGIN QUESTION
name: q3d
points: 1
manual: true
-->

**Answer:** *Double-click to edit with your answer*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->
<div class="alert alert-block alert-info">
<b>Question 3e:</b> What is the importance of differential false positive and false negative rates across race categories in regard to ProPublica’s analysis? How do these observations have real impacts on the defendants who receive scores?
</div>

<!--
BEGIN QUESTION
name: q3e
points: 1
manual: true
-->

**Answer:** *Double-click to edit with your answer*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->
<div class="alert alert-block alert-info">
<b>Question 3f (ii):</b> Justify to yourself why ProPublica's definition of fairness is violated by these FP/FN rates. What would the FP/FN rates have to look like for ProPublica to find COMPAS technically fair under this definition?
</div>

<!--
BEGIN QUESTION
name: q3f
points: 1
manual: true
-->

**Answer:** *Double-click to edit with your answer*

<!-- END QUESTION --> 

## 4. equivant's Analysis

### Can a model be racially biased without a race feature?
equivant made it very clear that they did not utilize any race feature (a column that informs the algorithm of the race of each defendant) in their dataset when they were training their algorithm. As a result of neglecting this race feature, equivant argued that the COMPAS tool would not take race into account when making decisions for defendants and therefore cannot be biased against certain races. 

This brings into question the possibility of a algorithm to infer, or proxy, features that aren't included in a dataset as it makes predictions. We often notice the ability of algorithms to proxy for omitted features via features included in the dataset when it happens with particularly sensitive or personally-identifying characteristics.

<!-- BEGIN QUESTION -->

<div class="alert alert-block alert-info">
<b>Question 4a:</b> Take a look back at the <a href="https://www.documentcloud.org/documents/2702103-Sample-Risk-Assessment-COMPAS-CORE.html">sample questionnaire</a> that is used by COMPAS to assign risk scores. Can you think of ways in which data captured in this questionnaire might serve as a proxy for features that aren't explicitly included, such as race?
</div>


<!--
BEGIN QUESTION
name: Q4a
points: 1
manual: true
-->

**Answer:** *Double-click to edit with your answer*

<!-- END QUESTION -->

<div class="alert alert-block alert-info">
<b>Question 4b:</b> Can data scientists’ algorithms be absolved of racial bias if they decide to not take race into account within their models? Explain why or why not.
</div>


<!--
BEGIN QUESTION
name: Q4b
points: 1
manual: true
-->

**Answer:** *Double-click to edit with your answer*

<!-- END QUESTION -->

Omitting race as an explicit feature in a dataset does not prevent algorithms from disproportionately affecting certain racial groups over others. Instead, removing features like race from a dataset obscures the need for conversations about the significance of race within the algorithm and the social context it operates in, while bolstering an algorithm's appearance of objectivity and impartiality.

#### Accuracy Equity™
Instead of analyzing and comparing the model errors FNR and FPR, Northpointe utilized the complement of FNR, known as the True Positive Rate (TPR) but often referred to as **sensitivity**, and paired it with the FPR in a visualization called an **ROC Curve**. Northpointe made use of the ROC Curve, which we describe below, to prove what they refer to as "Accuracy Equity." Accuracy equity, according to [Northpointe](http://go.volarisgroup.com/rs/430-MBX-989/images/ProPublica_Commentary_Final_070616.pdf), is exhibited in the model “if it can discriminate recidivists and nonrecidivists equally well for two different groups such as blacks and whites.” In order to connect these two ideas, we first need to learn what this curve does and how it is can be used to understand fairness.

In data science, we often use a visualization called the **Receiver Operating Characteristic (ROC) curve** to interpret how much a model is capable of distinguishing between groups. We do this by plotting the curve and deriving the **Area Under the Curve (AUC)**. The AUC must fall between 0 and 1. The higher the AUC, the better the model is at predicting our target variable. A model with an AUC of 0.5 (shown in the green dotted line in the ROC below) distinguishes between classes with about the same success as a coint toss (50-50 guess). Northpointe compared the AUC for Black and white defendants, comparing COMPAS' ability to accurately predict recidivism among Black and white defendants separately.

Below, we replicate the code they would have used to create an ROC curve comparing COMPAS's performance with Black and white defendants. You do not need to understand the code itself, but feel free to go through it if you are interested!

In [ ]:
# Calculate FPR and FNR from metrics package - Black defendants
black_def = propublica_data[propublica_data["race"] == "African-American"]
# True values (if the defendants reoffended)
y1 = black_def['two_year_recid']
# Predicted values from COMPAS tool (risk scores)
pred1 = black_def["decile_score"].replace([1, 2, 3, 4], 0).replace([5, 6, 7, 8, 9, 10], 1)
fpr_black, tpr_black, threshold = roc_curve(y1, pred1)

# Calculate FPR and FNR from metrics package - White defendants
white_def = propublica_data[propublica_data["race"] == "Caucasian"]
# True values (if the defendants reoffended)
y2 = white_def['two_year_recid']
# Predicted values from COMPAS tool (risk scores)
pred2 = white_def["decile_score"].replace([1, 2, 3, 4], 0).replace([5, 6, 7, 8, 9, 10], 1)
fpr_white, tpr_white, threshold = roc_curve(y2, pred2)

# Set up the plot for the ROC curve
plt.subplots(1, figsize=(10,10))
plt.title('ROC - Black Defendants')
plt.plot(fpr_black, tpr_black)
plt.plot(fpr_white, tpr_white)
plt.plot([0, 1], ls="--")
plt.ylabel('Sensitivity')
plt.xlabel('1 - Specificity')
plt.show()

In [ ]:
# Calculate and print the AUC
print("AUC for Black defendants:", roc_auc_score(y1, pred1))
print("AUC for white defendants:", roc_auc_score(y2, pred2))

<!-- BEGIN QUESTION -->
<div class="alert alert-block alert-info">
<b>Question 4c:</b> Why would equivant argue, on the basis of the ROC curve and AUC calculations above, that COMPAS is fair? Recall: the AUC helps us determine by how much COMPAS is capable of distinguishing between future recidivists and non-recidivists.
</div>

<!--
BEGIN QUESTION
name: q4c
points: 1
manual: true
-->

**Answer:** *Double-click to edit with your answer*

<!-- END QUESTION -->

Regardless of whether or not equivant argues that the ROC demonstrates COMPAS' fairness, it is worth pointing COMPAS is not particularly accurate. Given that the green dotted line (AUC of 0.5) represents a model which correctly predicts whether someone will commit a future crime 50% of the time, COMPAS's accuracy at predicting recidivism is markedly closer to that of a coin toss than a model which makes perfect predictions.

<!-- END QUESTION -->

## 5. Conclusion
### Which definition is fair? Who decides?
In the case of COMPAS, we have shown that ProPublica and equivant both used valid data science techniques to quantitatively demonstrate why the model is fair or unfair under competing definitions. Decisions of how to define complex and shifting concepts like fairness are frequently made by data scientists, and these decisions have serious real-world effects on the people subject to them.

<!-- BEGIN QUESTION -->
<div class="alert alert-block alert-info">
<b>Question 5a:</b> Why does removing race as a feature allow Northpointe to be percieved as unbiased? Considering situating your response within the context of current political events.</div>
 
<!--
BEGIN QUESTION
name: q5a
points: 1
manual: true
-->

**Answer:** *Double-click to edit with your answer*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->
<div class="alert alert-block alert-info">
<b>Question 5b:</b> Take a step back and think about how different actors who created, utilize, and are affected by COMPAS would consider which definition is most fair. Name two relevant actors, and discuss how they might justify their determination of which definition of fairness COMPAS should follow. If you don't think they'd choose either definition, explain why.
</div>

<!--
BEGIN QUESTION
name: q5b
points: 1
manual: true
-->

**Answer:** *Double-click to edit with your answer*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class="alert alert-block alert-info">
<b>Labor Question:</b> How much time did you spend completing this module? Did you find outside resources that helped you? if so what were they?
</div>


<!--
BEGIN QUESTION
name: QL
points: 1
manual: true
-->

<!-- END QUESTION -->

You’ve now begun to think through the very complex systems in which the COMPAS algorithm functions. Congratulations! Through considering the possibility of multiple definitions of fairness connected to COMPAS, hopefully you can begin to understand some of the human contexts of  creating algorithms that, intentionally or not, affect people and their decision-making. 
    
**Congratulations on finishing Notebook 1!**

The cell below generates a link to download your notebook as a zip file, which you can then submit.

**Before downloading, run all cells and save the notebook using `command`/`control` + `s` *or* clicking the save icon in the toolbar at the top of notebook. This is very important to ensure that all of your work shows up in the downloaded file!**
To submit your notebook, 

In [ ]:
# Run all above cells, save your notebook, and then run this cell to export your submission.
# Download the zip file, which contains a copy of your notebook and your written responses.
generator.export("notebook4.ipynb")